In [1]:
from pyspark import SparkContext
sc = SparkContext('local[*]', 'pyspark tutorial')
from time import sleep, time

# A decorator: timer 
https://codereview.stackexchange.com/questions/169870/decorator-to-measure-execution-time-of-a-function

In [43]:
from time import time
def timer(f):
    def wrapper(*args, **kwargs):  
        # 1. what are *args and **kwargs  ?  
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print 'The function {0} took {1} seconds to return.'.format(f.__name__, end-start)
        # 2. what is .format ?
        # 3. what is f.__name__ ?
        return result
    return wrapper
# issue: print command is executed on every recursive call of f (ideally it should be executed only once at the end)    

1. https://www.programiz.com/python-programming/args-and-kwargs
   - args (non keyword arguments) is a list 
   - kwargs (keyword arguments) is a dictionary 
2. https://pyformat.info/
3. https://stackoverflow.com/questions/251464/how-to-get-a-function-name-as-a-string-in-python

# Evaluating a function on multiple inputs in parallel

In [34]:
_limit = 8
my_list = range(_limit)

def fun(x):
    sleep(5)
    return x*x

@timer
def evaluate_parallel(my_list):
    my_rdd = sc.parallelize(my_list)
    my_rdd_transformed = my_rdd.map(lambda x: fun(x)).collect()
    return list(my_rdd_transformed)

In [33]:
evaluate_parallel(my_list)

the function evaluate_parallel took 10.0906891823 time


[0, 1, 4, 9, 16, 25, 36, 49]

# Apply a filter on a parallel collection

In [45]:
from random import random
_num_samples = 1000

def is_inside(p):
    sleep(0.01)
    x, y = random(), random()
    return x*x + y*y < 1

@timer
def get_pi():
    count = sc.parallelize(xrange(0, _num_samples)).filter(is_inside).count() # 1. what is xrange ?
    print "Pi is roughly %f" % (4.0 * count / _num_samples)

1. range vs xrange
    - https://www.geeksforgeeks.org/range-vs-xrange-python/
    - https://stackoverflow.com/questions/94935/what-is-the-difference-between-range-and-xrange-functions-in-python-2-x

In [46]:
get_pi()

Pi is roughly 3.120000
The function get_pi took 2.86020493507 seconds to return.


# The pagerank algorithm 
- https://apache.googlesource.com/spark/+/master/examples/src/main/python/pagerank.py
- https://developers.soundcloud.com/blog/pagerank-in-spark
- https://spark.apache.org/docs/latest/quick-start.html
- https://jaceklaskowski.gitbooks.io/mastering-apache-spark/content/spark-rdd-caching.html

In [47]:
my_rdd = sc.parallelize(my_list)

In [82]:
lines = sc.textFile('graph.txt')
# 1. what is the difference between sc.parallelize() and sc.textFile() ?

1. sc.textFile creates a parallel collection similar to parallelize()
   - https://stackoverflow.com/questions/44860973/what-are-the-differences-between-sc-parallelize-and-sc-textfile
   - https://www.quora.com/Is-my-understanding-to-parallel-operations-in-spark-correct
   - https://www.learningjournal.guru/article/apache-spark/apache-spark-parallel-processing/

In [73]:
def get_pair(line):
    line = line.strip()
    return line.split()[0], line.split()[1]

links = lines.map(lambda line: get_pair(line)).distinct().groupByKey().cache()
links.collect()

[(u'1', <pyspark.resultiterable.ResultIterable at 0x1163356d0>),
 (u'3', <pyspark.resultiterable.ResultIterable at 0x116337f50>),
 (u'2', <pyspark.resultiterable.ResultIterable at 0x116362050>),
 (u'4', <pyspark.resultiterable.ResultIterable at 0x1163620d0>)]

In [74]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))
ranks.collect()

[(u'1', 1.0), (u'3', 1.0), (u'2', 1.0), (u'4', 1.0)]

In [85]:
contribs = links.join(ranks)
contribs.collect()

[(u'2',
  (<pyspark.resultiterable.ResultIterable at 0x116352dd0>,
   0.23153811726092793)),
 (u'4',
  (<pyspark.resultiterable.ResultIterable at 0x116352fd0>,
   0.3299418170968223)),
 (u'1',
  (<pyspark.resultiterable.ResultIterable at 0x116335c10>,
   0.38370878711024914)),
 (u'3',
  (<pyspark.resultiterable.ResultIterable at 0x116335710>,
   0.3299418170968223))]

In [84]:
_num_iterations = 10

from operator import add

def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

for iteration in range(_num_iterations):
        # Calculates URL contributions to the rank of other URLs.  
        contribs = links.join(ranks).flatMap(
            lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
        # 1. what is flatMap ?
        
        # Re-calculates URL ranks based on neighbor contributions.
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

        
# Collects all URL ranks and dump them to console.
for (link, rank) in ranks.sortBy(lambda x: -x[1]).collect():
        print("%s has rank: %s." % (link, rank))
    
# 2. How to sort by ranks ?    
# 3. How to sort by values in descending order ?

1 has rank: 0.38370878711.
10 has rank: 0.371763389527.
3 has rank: 0.329941817097.
4 has rank: 0.329941817097.
6 has rank: 0.243483514844.
5 has rank: 0.243483514844.
2 has rank: 0.231538117261.


1. http://www.brunton-spall.co.uk/post/2011/12/02/map-map-and-flatmap-in-scala/
2. https://stackoverflow.com/questions/33706408/how-to-sort-by-value-efficiently-in-pyspark
3. https://stackoverflow.com/questions/30787635/takeordered-descending-pyspark